In [5]:
import pandas as pd

df_metro = pd.read_excel('metro_20251112-oficio-4770_2013.xlsx', sheet_name='Abiertos', skiprows=7)
df_metro.head()

,CODIGO,ENTIDAD,NOMBRE FANTASIA,DIRECCION,COMUNA,HORARIO REFERENCIAL,ESTE,NORTE,LONGITUD,LATITUD
0,L1,METRO,SAN PABLO L1,Av. San Pablo 6190,LO PRADO,"Lunes a Viernes: 6:00 a 23:00 horas, Sábados: ...",339892,6298155,-70.723068,-33.444355
1,L1,METRO,NEPTUNO,Av. Neptuno Oriente frente al 260,LO PRADO,"Lunes a Viernes: 6:00 a 23:00 horas, Sábados: ...",339903,6297368,-70.723086,-33.451453
2,L1,METRO,PAJARITOS,Av. General Bonilla 5900,LO PRADO,"Lunes a Viernes: 6:00 a 23:00 horas, Sábados: ...",340738,6296735,-70.714218,-33.457279
3,L1,METRO,LAS REJAS,Av. Libertador Bernardo O`Higgins 6056,ESTACION CENTRAL,"Lunes a Viernes: 6:00 a 23:00 horas, Sábados: ...",341466,6296689,-70.706398,-33.457802
4,L1,METRO,ECUADOR,Av. Libertador Bernardo O`Higgins Frente al 4620,ESTACION CENTRAL,"Lunes a Viernes: 6:00 a 23:00 horas, Sábados: ...",342093,6296850,-70.699623,-33.456448


In [7]:
df_metro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143 entries, 0 to 142
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CODIGO               143 non-null    object 
 1   ENTIDAD              143 non-null    object 
 2   NOMBRE FANTASIA      143 non-null    object 
 3   DIRECCION            143 non-null    object 
 4   COMUNA               143 non-null    object 
 5   HORARIO REFERENCIAL  143 non-null    object 
 6   ESTE                 143 non-null    int64  
 7   NORTE                143 non-null    int64  
 8   LONGITUD             143 non-null    float64
 9   LATITUD              143 non-null    float64
dtypes: float64(2), int64(2), object(6)
memory usage: 11.3+ KB


In [8]:
df_metro.columns

Index(['CODIGO', 'ENTIDAD', 'NOMBRE FANTASIA', 'DIRECCION', 'COMUNA',
       'HORARIO REFERENCIAL', 'ESTE', 'NORTE', 'LONGITUD', 'LATITUD'],
      dtype='object')

In [9]:
df_metro.shape

(143, 10)

In [14]:
cols_estaciones = ["CODIGO","NOMBRE FANTASIA","DIRECCION","COMUNA",'LONGITUD', 'LATITUD']
cols_estaciones = [c for c in cols_estaciones if c in df_metro.columns]

In [17]:
estaciones = (
    df_metro[cols_estaciones]
    .drop_duplicates(subset=["CODIGO", "NOMBRE FANTASIA", "COMUNA"])
    .sort_values(["COMUNA", "NOMBRE FANTASIA"])
)

In [18]:
estaciones.head(10)

,CODIGO,NOMBRE FANTASIA,DIRECCION,COMUNA,LONGITUD,LATITUD
133,L6,CERRILLOS,Martín De Solier 4598,CERRILLOS,-70.693596,-33.483799
57,L3,CARDENAL CARO,Av. José María Caro 1875,CONCHALI,-70.685539,-33.373062
59,L3,CONCHALI,Av. Independencia 3279,CONCHALI,-70.670349,-33.397388
58,L3,VIVACETA,Av. Independencia 4765,CONCHALI,-70.679634,-33.385063
51,L2,COPA LO MARTINEZ,Av. Padre Hurtado 12287,EL BOSQUE,-70.673638,-33.570657
52,L2,HOSPITAL EL PINO,Av. Padre Hurtado 13378,EL BOSQUE,-70.676324,-33.582233
50,L2,OBSERVATORIO,Av. Padre Hurtado 11828,EL BOSQUE,-70.670341,-33.560345
4,L1,ECUADOR,Av. Libertador Bernardo O`Higgins Frente al 4620,ESTACION CENTRAL,-70.699623,-33.456448
7,L1,ESTACION CENTRAL,Av. Libertador Bernardo O`Higgins Frente al 3322,ESTACION CENTRAL,-70.679977,-33.451984
3,L1,LAS REJAS,Av. Libertador Bernardo O`Higgins 6056,ESTACION CENTRAL,-70.706398,-33.457802


In [19]:
estaciones.to_csv("estaciones_metro.csv", index=False)

In [21]:
import folium

center_lat = estaciones.LATITUD.mean()
center_lon = estaciones.LONGITUD.mean()

m = folium.Map(location=[center_lat,center_lon], width="80%", height="80%")

for _, row in estaciones.iterrows():
  folium.CircleMarker(
      location=[row["LATITUD"],row["LONGITUD"]],
      radius=2,
      popup=f"{row['NOMBRE FANTASIA']} ({row['COMUNA']})",
  ).add_to(m)

m



In [22]:
colores = {
    "L1": "red",
    "L2": "orange",
    "L3": "brown",
    "L4": "blue",
    "L4A": "green",
    "L5": "darkgreen",
    "L6": "purple"
}

In [23]:
import folium

center_lat = estaciones["LATITUD"].mean()
center_lon = estaciones["LONGITUD"].mean()

m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=11,
    width="100%",
    height="80%"
)

In [24]:
for _, row in estaciones.dropna(subset=["LATITUD", "LONGITUD"]).iterrows():
    linea = row["CODIGO"]
    color = colores.get(linea, "gray")  # gris si es una línea no mapeada

    folium.CircleMarker(
        location=[row["LATITUD"], row["LONGITUD"]],
        radius=5,
        color=color,
        fill=True,
        fill_opacity=0.8,
        popup=f"{row['NOMBRE FANTASIA']} (Línea {linea})"
    ).add_to(m)

m

In [26]:
colores = {
    "L1": "red",
    "L2": "orange",
    "L3": "brown",
    "L4": "blue",
    "L4A": "green",
    "L5": "darkgreen",
    "L6": "purple"
}

m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=11,
    width="80%",
    height="80%"
)

for linea, grupo in estaciones.dropna(subset=["LATITUD", "LONGITUD"]).groupby("CODIGO"):
    capa = folium.FeatureGroup(name=f"Línea {linea}")
    color = colores.get(linea, "gray")

    for _, row in grupo.iterrows():
        folium.CircleMarker(
            location=[row["LATITUD"], row["LONGITUD"]],
            radius=5,
            color=color,
            fill=True,
            fill_opacity=0.8,
            popup=f"{row['NOMBRE FANTASIA']} (Línea {linea})"
        ).add_to(capa)

    m.add_child(capa)

folium.LayerControl().add_to(m)

m